The goal of this work is to implement, and test a method program that divides a dataset to a number of (maybe) overlapping subsets.
- Dataset 
- Distance 
- Homogeneity
- Decomposition
 - Baseline 
 - Our method

In [ ]:
print("hello world")

hello world


In [ ]:
# test lu

The link shared in Blackbord:
https://colab.research.google.com/drive/1tvxs0etNUDw1_8Aeu5G6y6qOIBROq-G6?usp=sharing#scrollTo=B7Gj19ZxzJGO


In [ ]:
!wget https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar -xvf spark-3.2.1-bin-hadoop3.2.tgz
!java -version

In [2]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os 
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

## You can add more config while building 
spark = SparkSession.builder.master("local[8]").\
                    config("spark.app.name","session_one").\
                    getOrCreate() #number of threads = 16

In [7]:
import pandas as pd

In [62]:
url="https://raw.githubusercontent.com/mustafa-majid/disfiles/main/cities.csv"
pdf=pd.read_csv(url)

pdf.head(5)

,LatD,"""LatM""","""LatS""","""NS""","""LonD""","""LonM""","""LonS""","""EW""","""City""","""State"""
0,41,5,59,"""N""",80,39,0,"""W""","""Youngstown""",OH
1,42,52,48,"""N""",97,23,23,"""W""","""Yankton""",SD
2,46,35,59,"""N""",120,30,36,"""W""","""Yakima""",WA
3,42,16,12,"""N""",71,48,0,"""W""","""Worcester""",MA
4,43,37,48,"""N""",89,46,11,"""W""","""Wisconsin Dells""",WI


In [67]:
p2=pd.read_csv(url).drop_duplicates()

In [68]:
p2.head(5)

,LatD,"""LatM""","""LatS""","""NS""","""LonD""","""LonM""","""LonS""","""EW""","""City""","""State"""
0,41,5,59,"""N""",80,39,0,"""W""","""Youngstown""",OH
1,42,52,48,"""N""",97,23,23,"""W""","""Yankton""",SD
2,46,35,59,"""N""",120,30,36,"""W""","""Yakima""",WA
3,42,16,12,"""N""",71,48,0,"""W""","""Worcester""",MA
4,43,37,48,"""N""",89,46,11,"""W""","""Wisconsin Dells""",WI


In [120]:
import urllib.request
import string
file = urllib.request.urlopen(url)
print(type(file))
li=[]
for line in file:
  s = line.decode("utf-8").translate(str.maketrans('', '', string.punctuation))
   
  li.append(" ".join(s.split()))

li.pop(0)
print(li[2])
print(type(li[2]))

<class 'http.client.HTTPResponse'>
46 35 59 N 120 30 36 W Yakima WA
<class 'str'>


In [36]:
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql import SQLContext, SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Word2Vec
from pyspark.sql.functions import *

In [122]:
from pyspark.sql import Row

rdd1 = sc.parallelize(li)
row_rdd = rdd1.map(lambda x: Row(x))
df=sqlContext.createDataFrame(row_rdd,['words'])
df.show(50,False)

+----------------------------------------+
|words                                   |
+----------------------------------------+
|41 5 59 N 80 39 0 W Youngstown OH       |
|42 52 48 N 97 23 23 W Yankton SD        |
|46 35 59 N 120 30 36 W Yakima WA        |
|42 16 12 N 71 48 0 W Worcester MA       |
|43 37 48 N 89 46 11 W Wisconsin Dells WI|
|36 5 59 N 80 15 0 W WinstonSalem NC     |
|49 52 48 N 97 9 0 W Winnipeg MB         |
|39 11 23 N 78 9 36 W Winchester VA      |
|34 14 24 N 77 55 11 W Wilmington NC     |
|39 45 0 N 75 33 0 W Wilmington DE       |
|48 9 0 N 103 37 12 W Williston ND       |
|41 15 0 N 77 0 0 W Williamsport PA      |
|37 40 48 N 82 16 47 W Williamson WV     |
|33 54 0 N 98 29 23 W Wichita Falls TX   |
|37 41 23 N 97 20 23 W Wichita KS        |
|40 4 11 N 80 43 12 W Wheeling WV        |
|26 43 11 N 80 3 0 W West Palm Beach FL  |
|47 25 11 N 120 19 11 W Wenatchee WA     |
|41 25 11 N 122 23 23 W Weed CA          |
|31 13 11 N 82 20 59 W Waycross GA       |
|44 57 35 N

In [123]:
df2 = df.withColumn("text_splitted", split(lower(col("words")), " "))

In [153]:
df2.show(20)

+--------------------+--------------------+
|               words|       text_splitted|
+--------------------+--------------------+
|41 5 59 N 80 39 0...|[41, 5, 59, n, 80...|
|42 52 48 N 97 23 ...|[42, 52, 48, n, 9...|
|46 35 59 N 120 30...|[46, 35, 59, n, 1...|
|42 16 12 N 71 48 ...|[42, 16, 12, n, 7...|
|43 37 48 N 89 46 ...|[43, 37, 48, n, 8...|
|36 5 59 N 80 15 0...|[36, 5, 59, n, 80...|
|49 52 48 N 97 9 0...|[49, 52, 48, n, 9...|
|39 11 23 N 78 9 3...|[39, 11, 23, n, 7...|
|34 14 24 N 77 55 ...|[34, 14, 24, n, 7...|
|39 45 0 N 75 33 0...|[39, 45, 0, n, 75...|
|48 9 0 N 103 37 1...|[48, 9, 0, n, 103...|
|41 15 0 N 77 0 0 ...|[41, 15, 0, n, 77...|
|37 40 48 N 82 16 ...|[37, 40, 48, n, 8...|
|33 54 0 N 98 29 2...|[33, 54, 0, n, 98...|
|37 41 23 N 97 20 ...|[37, 41, 23, n, 9...|
|40 4 11 N 80 43 1...|[40, 4, 11, n, 80...|
|26 43 11 N 80 3 0...|[26, 43, 11, n, 8...|
|47 25 11 N 120 19...|[47, 25, 11, n, 1...|
|41 25 11 N 122 23...|[41, 25, 11, n, 1...|
|31 13 11 N 82 20 ...|[31, 13, 1

In [126]:
word2Vec = Word2Vec( minCount=0, maxIter=100, inputCol="text_splitted", outputCol="features")
model = word2Vec.fit(df2)

In [128]:
result = model.transform(df2)

In [167]:
numIterations = 300
numberClusters = 30
kmeans = KMeans().setMaxIter(numIterations).setK(numberClusters).setSeed(1)
kmeans_model = kmeans.fit(result)

In [171]:
from pyspark.sql.functions import col

predictions = kmeans_model.transform(result)
predictions=predictions.orderBy(predictions.prediction.desc())
predictions = predictions.drop(col("features"))
predictions = predictions.drop(col("text_splitted"))
predictions.show(100,False)


+----------------------------------------+----------+
|words                                   |prediction|
+----------------------------------------+----------+
|47 25 11 N 120 19 11 W Wenatchee WA     |29        |
|46 4 11 N 118 19 48 W Walla Walla WA    |29        |
|36 9 35 N 95 54 36 W Tulsa OK           |28        |
|43 9 35 N 77 36 36 W Rochester NY       |28        |
|43 2 59 N 76 9 0 W Syracuse NY          |28        |
|39 11 23 N 78 9 36 W Winchester VA      |28        |
|43 6 36 N 75 13 48 W Utica NY           |28        |
|43 58 47 N 75 55 11 W Watertown NY      |28        |
|40 45 35 N 111 52 47 W Salt Lake City UT|27        |
|44 45 35 N 85 37 47 W Traverse City MI  |27        |
|38 46 12 N 112 5 23 W Richfield UT      |26        |
|42 49 11 N 73 56 59 W Schenectady NY    |25        |
|38 22 11 N 75 35 59 W Salisbury MD      |25        |
|37 32 24 N 77 26 59 W Richmond VA       |25        |
|37 16 12 N 79 56 24 W Roanoke VA        |25        |
|43 32 24 N 96 43 48 W Sioux

In [170]:
model.getVectors().show(20,False)

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [182]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

similar("Christian","Christine")

0.8888888888888888